In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder

# Definição da amostra

In [ ]:
popu = pd.read_csv("Popu_series.csv", encoding = "ISO-8859-1")
popu = popu.sample(frac=0.3, replace=False, random_state=195)
len(popu)

In [ ]:
popu = popu['serie']
popu

In [ ]:
popu.to_csv("Amostra.csv", index=False)

# Análise Exploratória

In [ ]:
#imdb = pd.read_csv("imdb_series.csv") REMOVER (arquivo da amostra antiga)
imdb = pd.read_csv("Amostra_IMDB.csv")
twitter = pd.read_csv("series_dataset_count/dataset_count.csv")
reviews = pd.read_csv("UserReviewsCritic.csv", sep=";")

In [ ]:
reviews.head()

In [ ]:
twitter.dropna(inplace=True)

In [ ]:
twitter

In [ ]:
imdb.head()

In [ ]:
reviews.rename(columns={"titulo":"Titulo"}, inplace=True)
aux = pd.merge(imdb, twitter, on = "IMDBid")
dados = pd.merge(aux, reviews, on = "Titulo")

#dados.drop(dados[dados["Nome"]=="LoveDeathandRobots"].index, inplace=True)
dados.head()

In [ ]:
dados.drop(columns="Reviews", inplace=True)

In [ ]:
indice = dados[dados['Duração']>100]['Duração'].index

In [ ]:
dados.loc[indice, "Duração"] = dados.loc[indice, "Duração"]/dados.loc[indice, "Episodios"]

In [ ]:
dados[dados['Duração']>100]

In [ ]:
dados['DataEstreia'] = pd.to_datetime(dados['Data de estréia'], format="%Y-%m-%d", errors = 'coerce')
dados['mes'] = pd.DatetimeIndex(dados['Data de estréia']).month

LEMBRAR DE DIVIDIR O ENGAJAMENTO POR NUMERO DE DIAS!!

In [ ]:
diasPre = 75-3
diasDurante = 7
diasPos = 150-3
diasTotal = 226
dados['TaxaPre'] = dados['CountPre']/diasPre
dados['TaxaDurante'] = dados['CountDurante']/diasDurante
dados['TaxaPos'] = dados['CountPos']/diasPos
dados['TaxaTotal'] = dados['CountTotal']/diasPos

In [ ]:
#dados[dados['Titulo']=="Julie and the Phantoms"]
dados.head()

### Hot encoding Gênero

In [ ]:
dados['Gênero'] = dados['Gênero'].apply(lambda x : x.split(", "))
dummies = pd.get_dummies(dados['Gênero'].apply(pd.Series).stack()).sum(level=0)
dados = dados.join(dummies)

# Análise Exploratória

In [ ]:
teste = dados[dados['Duração'].notna()]

In [ ]:
#dados.apply(lambda linha: linha['Duração'] if linha['Duração']<100 else linha['Duração']/linha['Episodios'])
#dados.apply(lambda linha: print(linha[7]))
#a = teste.apply(lambda linha: print(linha[0]))
#dados['Duração']

#duracao = dados['Duração']
#epi = dados['Episodios']


## Nota

In [ ]:
nota1 = sns.histplot(data=dados, x="Nota")

nota1.set_title('Histograma das notas no IMDb')
nota1.set_ylabel('Frequência')

In [ ]:
nota2 = sns.boxplot(y=dados['Nota'], data=dados)

nota2.set_title('Boxplot das notas no IMDb')
nota2.set_ylabel('Nota')

In [ ]:
nota3 = sns.boxplot(x=dados['Ano'],y=dados['Nota'], data=dados)

nota3.set_title('Boxplot das notas no IMDb por ano de lançamento')

In [ ]:
y = []
coluna = []
for i in range (24,dados.columns.size,1):
    coluna.append(str(dados.columns[i]))
    
    aux = []
    for j in range (0, len(dados[dados[coluna[-1]] == 1]['Nota']),1):
        aux.append(dados[dados[coluna[-1]] == 1]['Nota'].values[j])
        
    y.append(aux)

In [ ]:
colunas = {'Nota':[], 'Gênero':[]}
df_aux = pd.DataFrame(colunas)

for i in range(len(y)):
    for val in y[i]:
        linha = {'Nota':val, 'Gênero':coluna[i]}
        df_aux = df_aux.append(linha, ignore_index=True)

In [ ]:
df_aux ## PQ TA DANDO ERRO?

In [ ]:
fig, ax = plt.subplots()
nota4 = sns.boxplot(x='Gênero',y='Nota', data=df_aux)
ax.tick_params(rotation=90)

nota4.set_title('Boxplot das notas no IMDb por gênero')

#fig, ax = plt.subplots()
#sns.barplot(x = x, y = df['log_disaster'], palette="Blues_d")
#ax.tick_params(rotation=90)
#ax.set_title('Número de Desastres por Ano')
##ax.set_ylabel('Número de Desastres')
#ax.set_xlabel('Ano')
#fig.set_size_inches(20, 5)

In [ ]:
#["Mystery" in linha for linha in dados['Gênero']]
#plt.boxplot(dados[dados['Mystery']==1]['Nota'].describe())#.plot()
#plt.boxplot(dados[dados['Drama']==1]['Nota'].describe())#.plot()
#dados[dados['Drama']==1]['Nota'].describe()#.plot()

In [ ]:
#dados[dados['Drama']==1]['Nota'].count()

In [ ]:
#type(dados['Gênero'].apply(lambda x: x)[0])

In [ ]:

# Primeiramente: Fazer o count, ver a frequencia de cada série
# Segundamente: Avaliar os grupos mais frequentes (Drama+Comedia)

In [ ]:
dados.iloc[:, 24:].sum()
dados.iloc[:, 24:].sum()*100/86

### Nota do IMDB e número de votos por engajamento

In [ ]:
#dados['Nota'] CountPre CountDurante CountPos CountTotal

sns.scatterplot(data=dados, x="Nota", y="CountTotal")
#sns.regplot(data=dados[dados['Nota']>4], x="Nota", y="CountTotal")

#nota1.set_title('Histograma das notas no IMDb')
#nota1.set_ylabel('Frequência')

In [ ]:
sns.boxplot(data=dados, x="Nota", y="País")#, hue=dados['País'])


In [ ]:
maiorespaises = dados[ (dados['País']=="United States") | (dados['País']=="Brazil") | (dados['País']=="South Korea") | (dados['País']=="United Kingdom") | (dados['País']=="Germany") | (dados['País']=="India")]
sns.boxplot(data=maiorespaises, x="Nota", y="País")#, hue=dados['País'])


In [ ]:
sns.boxplot(data=dados[dados['País'].str.contains(",")==False], x="Nota", y="País")#, hue=dados['País'])

In [ ]:
sns.boxplot(data=dados, x="Episodios", y="Nota")

In [ ]:
tipo1 = sns.boxplot(data=dados, x="Check", y="Nota")
tipo1.set_title('Boxplot das notas de acordo com o tipo de série')
tipo1.set_xlabel('Tipo de série')

In [ ]:
sns.scatterplot(data=dados, x="Nota", y="CountPre")

In [ ]:
sns.scatterplot(data=dados, x="Nota", y="CountDurante")

In [ ]:
sns.scatterplot(data=dados, x="Nota", y="CountPos")

In [ ]:
dados.corr(method='pearson')
#dados.corr(method='spearman')

In [ ]:
dados.corr(method='spearman')

In [ ]:
#sns.scatterplot(data=dados, x="Ano", y="CountTotal") #countplot seria melhor?

In [ ]:
#sns.scatterplot(data=dados, x="Ano", y="Votos")#ja fizemos o boxplot em baixo

In [ ]:
sns.scatterplot(data=dados, x="Votos", y="CountTotal")

In [ ]:
sns.scatterplot(data=dados, x="Nota", y="Votos")

## Votos

In [ ]:
# 120000 - 20000 - 2500
votos1 = sns.histplot(data=dados, x="Votos")#, log_scale=True)

votos1.set_title('Histograma da quantidade de votos no IMDb')
votos1.set_ylabel('Frequência')

In [ ]:
votos2 = sns.histplot(data=dados[dados['Votos']<100000], x="Votos")#, log_scale=True)

votos2.set_title('Histograma da quantidade de votos no IMDb (<100.000)')
votos2.set_ylabel('Frequência')

In [ ]:
votos3 = sns.scatterplot(data=dados, x="Nota", y="Votos")
#sns.regplot(data=dados[dados['Nota']>4], x="Nota", y="Votos")

votos3.set_title('Diagrama de dispersão da nota e quantidade de votos no IMDb')


In [ ]:
votos4 = sns.boxplot(data=dados, x="Ano", y="Votos")
votos4.set_title('Boxplot da quantidade de votos no IMDb por ano de lançamento')

In [ ]:
votos5 = sns.boxplot(data=dados, x="Votos", y="País")
votos5.set_title('Boxplot da quantidade de votos por país de origem')

In [ ]:
maiorespaises = dados[ (dados['País']=="United States") | (dados['País']=="Brazil") | (dados['País']=="South Korea") | (dados['País']=="United Kingdom") | (dados['País']=="Germany") | (dados['País']=="India")]
votos5 = sns.boxplot(data=maiorespaises, x="Votos", y="País")
votos5.set_title('Boxplot da quantidade de votos por país de origem (mais populares)')

In [ ]:
votos5b = sns.boxplot(data=dados[dados['País'].str.contains(",")==False], x="Votos", y="País")
votos5b.set_title('Boxplot da quantidade de votos por país de origem (mais populares)')


In [ ]:
votos6 = sns.boxplot(data=dados, x="Episodios", y="Votos")
votos6.set_title('Boxplot da quantidade de votos por número de episódios')

In [ ]:
#sns.scatterplot(data=dados, x="user_reviews", y="Votos")
votos7 = sns.regplot(data=dados, x="user_reviews", y="Votos")
votos7.set_title('Diagrama de dispersão da quantidade de votos por quantidade de reviews dos usuários do IMDb')
votos7.set_xlabel('Reviews dos usuários do IMDb')

In [ ]:
#sns.scatterplot(data=dados, x="critic_reviews", y="Votos")
votos8 = sns.regplot(data=dados, x="critic_reviews", y="Votos")
votos8.set_title('Diagrama de dispersão da quantidade de votos por quantidade de reviews dos críticos do IMDb')
votos8.set_xlabel('Reviews dos críticos do IMDb')

# Duração

In [ ]:
dura1 = sns.histplot(data=dados, x="Duração")
dura1.set_title('Histograma da duração média dos episódios por série')
dura1.set_xlabel('Frequência')

In [ ]:
dura2=sns.boxplot(data=dados, x="Duração")
dura2.set_title('Boxplot da duração média dos episódios por série')

In [ ]:
sns.scatterplot(data=dados, x="Duração", y="Votos")

In [ ]:
dura3=sns.boxplot(data=dados, x="Ano", y="Duração")
dura3.set_title('Boxplot da duração média de episódios por ano de lançamento da série')

In [ ]:
dura4=sns.boxplot(data=dados, x="Check", y="Duração")
dura4.set_title('Boxplot da duração média de episódios por tipo de série')
dura4.set_xlabel('Tipo de série')

# Ano

In [ ]:
ano1 = sns.countplot(data=dados, x="Ano")# tratar (dividir duracao por numero de episodios para >100)
ano1.set_title('Gráfico de barras do ano de lançamento das séries')
ano1.set_ylabel('Frequência')

In [ ]:
sns.boxplot(data=dados, x="Ano")# tratar (dividir duracao por numero de episodios para >100)

# Data de estréia

In [ ]:
mes1 = sns.countplot(data=dados, x="mes")
mes1.set_title('Quantidade de séries por mês de lançamento')
mes1.set_xlabel('Mês')
mes1.set_ylabel('Frequência')

# País

In [ ]:
dados['País'].value_counts()

In [ ]:
fig, ax = plt.subplots()
sns.countplot(data=dados[dados['País'].str.contains(",")==False], x="País")
ax.tick_params(rotation=90)


# Idiomas

In [ ]:
dados['Idiomas'].value_counts()

In [ ]:
sns.countplot(data=dados, x="Idiomas", order=dados.Idiomas.value_counts().iloc[:7].index)

#visCausaMorte = sns.countplot(x="CAUSABAS", order=dados.CAUSABAS.value_counts().iloc[:10].index, data=dados, palette="Set1")

In [ ]:
fig, ax = plt.subplots()
sns.countplot(data=dados[dados['Idiomas'].str.contains(",")==False], x="Idiomas")
ax.tick_params(rotation=90)

# Episodios

In [ ]:
epis1 = sns.countplot(data=dados, x="Episodios")
epis1.set_title('Gráfico de barras do número de episódios')
epis1.set_ylabel('Frequência')

In [ ]:
epis2 = sns.boxplot(data=dados, x="Check", y="Episodios")
epis2.set_title('Boxplot do número de episódios por tipo de série')
epis2.set_ylabel('Frequência')

## Reviews de usuários

In [ ]:
rev1 = sns.histplot(data=dados, x="user_reviews")
rev1.set_title('Histograma do número de reviews de usuários no IMDb')
rev1.set_ylabel('Frequência')
rev1.set_xlabel('Reviews dos usuários no IMDb')

In [ ]:
rev2 = sns.boxplot(data=dados, x = "Ano",y="user_reviews")
rev2.set_title('Boxplot do número de reviews de usuários no IMDb por ano de lançamento')
rev2.set_ylabel('Reviews dos usuários no IMDb')

## Reviews de críticos

In [ ]:
rev3 = sns.histplot(data=dados, x="critic_reviews")

rev3.set_title('Histograma do número de reviews de críticos no IMDb')
rev3.set_xlabel('Reviews dos críticos no IMDb')
rev3.set_ylabel('Frequência')

In [ ]:
rev4 = sns.boxplot(data=dados, x = "Ano",y="critic_reviews")

rev4.set_title('Boxplot do número de reviews de críticos no IMDb por ano de lançamento')
rev4.set_ylabel('Reviews dos críticos no IMDb')

# Engajamento (counts)

## Histogramas do números de tweets em difentes fases (Total, Pré, Durante, Pós)

TODO: Agrupas esses gráficos 2x2

In [ ]:
countTotal1 = sns.histplot(data=dados, x="CountTotal")
countTotal1.set(xlabel='Número total de tweets', ylabel='Número séries', title='Histograma Número total de Tweets - Todas fases')

In [ ]:
countPre1 = sns.histplot(data=dados, x="CountPre")
countPre1.set(xlabel='Número total de tweets', ylabel='Número séries', title='Histograma Número total de Tweets - Fase Pré')

In [ ]:
countDurante1 = sns.histplot(data=dados, x="CountDurante")
countDurante1.set(xlabel='Número total de tweets', ylabel='Número séries', title='Histograma Número total de Tweets - Fase Durante')


In [ ]:
countPos1 = sns.histplot(data=dados, x="CountPos")
countPos1.set(xlabel='Número total de tweets', ylabel='Número séries', title='Histograma Número total de Tweets - Fase Pós')

dados[dados.CountPos>100000]


### Engajamento por ano (Total, Pré, Durante, Pós)

TODO: Agrupar gráficos

In [ ]:
countTotal2 = sns.boxplot(data=dados, x="Ano", y="CountTotal")
countTotal2.set(xlabel='Ano', ylabel='Número tweets', title='Total de Tweets')

In [ ]:
countPre2 = sns.boxplot(data=dados, x="Ano", y="CountPre")
countPre2.set(xlabel='Ano', ylabel='Número tweets', title='Total de Tweets - Fase pré')


In [ ]:
countDurante2 = sns.boxplot(data=dados, x="Ano", y="CountDurante")
countDurante2.set(xlabel='Ano', ylabel='Número tweets', title='Total de Tweets - Fase Durante')

In [ ]:
countPos2 = sns.boxplot(data=dados, x="Ano", y="CountPos")
countPos2.set(xlabel='Ano', ylabel='Número tweets', title='Total de Tweets - Fase Pós')

In [ ]:
dados[(dados.CountTotal>80000) & (dados.Ano == 2020) ]

Aparentemente, a utilização do Twitter para comentar séries vem diminuindo ao longo dos anos com exceção de algumas séries:
- Bridgeton (esperado já que foi uma das séries mais assistida da Netflix de todos os anos)
- Julie and Phantoms (essa série parece estar errada de alguma forma -> ela não é famosa mas tem muito tweets)???

In [ ]:
def getCount(tipocount):
    y = []
    coluna = []
    for i in range (24,dados.columns.size,1):
        coluna.append(str(dados.columns[i]))
        
        aux = []
        for j in range (0, len(dados[dados[coluna[-1]] == 1][tipocount]),1):
            aux.append(dados[dados[coluna[-1]] == 1][tipocount].values[j])
            
        y.append(aux)

    colunas = {'Count':[], 'Gênero':[]}
    df_aux = pd.DataFrame(colunas)

    for i in range(len(y)):
        for val in y[i]:
            linha = {'Count':val, 'Gênero':coluna[i]}
            df_aux = df_aux.append(linha, ignore_index=True)

    return df_aux

dfTotal = getCount("CountTotal")



In [ ]:
dfTotal

### Número de Tweets vs. Gênero

TODO: agrupar gráficos 4x4

In [ ]:
fig, ax = plt.subplots()
countTotal3 = sns.boxplot(x='Gênero',y='Count', data=dfTotal)
countTotal3.set(xlabel='Gênero', ylabel='Número tweets', title='Total de Tweets')
ax.tick_params(axis='x', rotation=90)

In [ ]:
dfPre = getCount("CountPre")
fig, ax = plt.subplots()
countPre3 = sns.boxplot(x='Gênero',y='Count', data=dfPre)
countPre3.set(xlabel='Gênero', ylabel='Número tweets', title='Total de Tweets - Fase Pré')
ax.tick_params(axis='x', rotation=90)

In [ ]:
dfDurante = getCount("CountDurante")
fig, ax = plt.subplots()
countDurante3 = sns.boxplot(x='Gênero',y='Count', data=dfDurante)
countDurante3.set(xlabel='Gênero', ylabel='Número tweets', title='Total de Tweets - Fase Durante')
ax.tick_params(axis='x', rotation=90)

In [ ]:
dfPos = getCount("CountPos")
fig, ax = plt.subplots()
countPos3 = sns.boxplot(x='Gênero',y='Count', data=dfPos)
countPos3.set(xlabel='Gênero', ylabel='Número tweets', title='Total de Tweets - Fase Pós')
ax.tick_params(axis='x', rotation=90)

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(18, 12), sharey=True)
fig.suptitle('Total de Tweets x Gênero', fontsize = 20)
plt.subplots_adjust(hspace = .45, wspace = 0.1)

sns.boxplot(ax=axes[0,0], x='Gênero',y='Count', data=dfTotal)
axes[0,0].set(xlabel=None, ylabel=None, title='Todas as Fases')
axes[0,0].tick_params(axis='x', rotation=90)

dfPre = getCount("CountPre")
sns.boxplot(ax=axes[0,1], x='Gênero',y='Count', data=dfPre)
axes[0,1].set(xlabel=None, ylabel=None, title='Fase Pré')
axes[0,1].tick_params(axis='x', rotation=90)

dfDurante = getCount("CountDurante")
sns.boxplot(ax=axes[1,0],x='Gênero',y='Count', data=dfDurante)
axes[1,0].set(xlabel=None, ylabel=None, title='Fase Durante')
axes[1,0].tick_params(axis='x', rotation=90)

dfPos = getCount("CountPos")
sns.boxplot(ax=axes[1,1], x='Gênero',y='Count', data=dfPos)
axes[1,1].set(xlabel=None, ylabel=None, title='Fase Pós')
axes[1,1].tick_params(axis='x', rotation=90)


### Número de tweets vs. mês de estréia

In [ ]:
countTotal4 = sns.boxplot(data=dados, x="mes", y="CountTotal")
countTotal4.set(xlabel='Gênero', ylabel='Número tweets', title='Boxplot Total de Tweets / mês')

In [ ]:
countPre4 = sns.boxplot(data=dados, x="mes", y="CountPre")
countPre4.set(xlabel='Gênero', ylabel='Número tweets', title='Boxplot Total de Tweets / mês - Fase pré')

In [ ]:
countDurante4 = sns.boxplot(data=dados, x="mes", y="CountDurante")
countDurante4.set(xlabel='Gênero', ylabel='Número tweets', title='Boxplot Total de Tweets / mês - Fase Durante')

In [ ]:
sns.boxplot(data=dados, x="mes", y="CountPos")

In [ ]:
sns.boxplot(data=dados, x="Check", y="CountTotal")

In [ ]:
sns.boxplot(data=dados, x="Episodios", y="CountTotal")

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=dados, x="País", y="CountTotal")
ax.tick_params(rotation=90)

In [ ]:
fig, ax = plt.subplots()
sns.boxplot(data=dados, x="Idiomas", y="CountTotal")
ax.tick_params(rotation=90)

# Série Temporal do engajamento

### Número e duração de episódios por engajamento

In [ ]:
sns.boxplot(data=dados, x="Episodios", y="CountTotal")

In [ ]:
sns.scatterplot(data=dados, x="Duração", y="CountTotal")

In [ ]:
dados[dados['Duração']>100] #corrigir, pegar a media por episodio (duracao/episodios)

## Atores e engajamento

## Influência de engajamento anterior no posterior

In [ ]:
sns.scatterplot(data=dados, x="CountPre", y="CountDurante")

In [ ]:
sns.scatterplot(data=dados, x="CountDurante", y="CountPos")

## Engajamento e genêro

In [ ]:
# dados['Gênero'] = dados['Gênero'].apply(lambda x : x.split(", "))
# dados['Gênero']

In [ ]:
# pd.get_dummies(dados['Gênero'].apply(pd.Series).stack()).sum(level=0)
# # Primeiramente: Fazer o count, ver a frequencia de cada série
# Segundamente: Avaliar os grupos mais frequentes (Drama+Comedia)

## Censura e engajamento

## Reviews e engajamento